In [ ]:
from utils.text_to_speech import TextToSpeech, VOICES
import os
import pandas as pd
from tqdm import tqdm
import yaml

In [ ]:
CATEGORY = yaml.safe_load(open("STEP_0_define_category_sub_category.yaml", "r"))['category']
SUB_CATEGORY = yaml.safe_load(open("STEP_0_define_category_sub_category.yaml", "r"))['sub_category']
print(f"Category: {CATEGORY}")
print(f"Sub Category: {SUB_CATEGORY}")

In [ ]:
# first create a dataframe for the text
df = pd.read_csv(f"category_data/individual_dataframes/{CATEGORY}_{SUB_CATEGORY}.csv")

In [ ]:
len(df), df.columns

In [ ]:
tts = TextToSpeech()

In [ ]:
json_obj = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    obj = {}
    for lang in 'english', 'italian':
        obj[lang] = {
            "text": row[lang],
        }
        synth_obj = tts.synthesize(
            text=row[lang],
            voice_name=VOICES[lang]['male'],
            speaking_rate=0.75 if lang == 'italian' else 1.0,
            pitch=0,
            verbose=False,
        )
        obj[lang]['hash'] = synth_obj['hash']
        obj[lang]['audio_file'] = synth_obj['audio_file']
        obj[lang]['voice_name'] = synth_obj['voice_name']
        obj[lang]['speaking_rate'] = synth_obj['speaking_rate']
        obj[lang]['pitch'] = synth_obj['pitch']
    for key in row.keys():
        if key not in ['english', 'italian']:
            if str(row[key]) == 'nan':
                obj[key] = ""
            else:
                obj[key] = row[key]

    json_obj.append(obj)

In [ ]:
json_obj

In [ ]:
import json
json.dump(
    json_obj,
    open(f"category_data/step_1/{CATEGORY}_{SUB_CATEGORY}.json", "w"),
    indent=2,
    ensure_ascii=False,
)